# **Download Cleaning Dataset**

In [51]:
import pandas as pd

#load clean dataset
df = pd.read_csv('/content/cleaned_reviews_data.csv')

In [52]:
df.head()

,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpfulness_ratio,date,full_text
0,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1.0,2011-04-27,Good Quality Dog Food I have bought several of...
1,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0.0,2012-09-07,Not as Advertised Product arrived labeled as J...
2,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1.0,2008-08-18,"""Delight"" says it all This is a confection tha..."
3,B000UA0QIQ,A395BORC6FGVXV,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,1.0,2011-06-13,Cough Medicine If you are looking for the secr...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,0.0,2012-10-21,Great taffy Great taffy at a great price. The...


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88916 entries, 0 to 88915
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ProductId               88916 non-null  object 
 1   UserId                  88916 non-null  object 
 2   HelpfulnessNumerator    88916 non-null  int64  
 3   HelpfulnessDenominator  88916 non-null  int64  
 4   Score                   88916 non-null  int64  
 5   Time                    88916 non-null  int64  
 6   Summary                 88916 non-null  object 
 7   Text                    88916 non-null  object 
 8   helpfulness_ratio       88916 non-null  float64
 9   date                    88916 non-null  object 
 10  full_text               88916 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 7.5+ MB


# **Preprocessing**

**Label Encoding**

In [54]:
def sentiment_label(score):
    if score <= 2:
        return 0   # Negative
    elif score <= 4 :
        return 1   # Neutral
    else:
        return 2   # Positive

df["sentiment"] = df["Score"].apply(sentiment_label)
df["sentiment"].value_counts()

,count
sentiment,
2,55477
1,20119
0,13320


In [55]:
# Use full_text as input text
TEXT_COL = "full_text"

# Check result
df[[TEXT_COL, "Score", "sentiment"]].head()

,full_text,Score,sentiment
0,Good Quality Dog Food I have bought several of...,5,2
1,Not as Advertised Product arrived labeled as J...,1,0
2,"""Delight"" says it all This is a confection tha...",4,1
3,Cough Medicine If you are looking for the secr...,2,0
4,Great taffy Great taffy at a great price. The...,5,2


# **TEXT PREPROCESSING**

**Negation Handling**

In [56]:
import re

def handle_negation(text):
    """
    Combine negation with the following word:
    not good -> not_good
    not bad  -> not_bad
    """
    tokens = text.split()
    result = []
    i = 0
    while i < len(tokens):
        if tokens[i] == "not" and i + 1 < len(tokens):
            result.append("not_" + tokens[i + 1])
            i += 2
        else:
            result.append(tokens[i])
            i += 1
    return " ".join(result)

def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = handle_negation(text)   # 🔥 Negation Handling
    return text

df["clean_text"] = df[TEXT_COL].astype(str).apply(clean_text)

print("Before:\n", df[TEXT_COL].iloc[0])
print("\nAfter:\n", df["clean_text"].iloc[0])

Before:
 Good Quality Dog Food I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

After:
 good quality dog food i have bought several of the vitality canned dog food products and have found them all to be of good quality the product looks more like a stew than a processed meat and it smells better my labrador is finicky and she appreciates this product better than most


**Train / Test Split**

In [57]:
from sklearn.model_selection import train_test_split
import numpy as np

X = df["clean_text"]
y = df["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

y_train = np.array(y_train)
y_test  = np.array(y_test)

**Tokenization + Padding**

In [58]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

VOCAB_SIZE = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad  = pad_sequences(X_test_seq,  maxlen=MAX_LEN, padding="post")

X_train_pad.shape, X_test_pad.shape

((71132, 100), (17784, 100))

# **GRU Model**

In [59]:
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [60]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

EMBEDDING_DIM = 128
NUM_CLASSES = 3
BATCH_SIZE = 64
EPOCHS = 50

In [61]:
gru_model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_shape=(MAX_LEN,)),
    GRU(64),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation="softmax")
])

gru_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

gru_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,317,443 (5.03 MB)

 Trainable params: 1,317,443 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
start_time = time.time()

gru_model.fit(
    X_train_pad,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

gru_time = time.time() - start_time

Epoch 1/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.6255 - loss: 0.9055 - val_accuracy: 0.7372 - val_loss: 0.6069
Epoch 2/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.7700 - loss: 0.5509 - val_accuracy: 0.7934 - val_loss: 0.5048
Epoch 3/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.8365 - loss: 0.4087 - val_accuracy: 0.7994 - val_loss: 0.5085
Epoch 4/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8729 - loss: 0.3343 - val_accuracy: 0.7938 - val_loss: 0.5411
Epoch 5/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9010 - loss: 0.2713 - val_accuracy: 0.7874 - val_loss: 0.6167
Epoch 6/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9260 - loss: 0.2122 - val_accuracy: 0.7822 - val_loss: 0.6959
Epoch 7/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9435 - loss: 0.1691 - val_accuracy: 0.7731 - val_loss: 0.8320


In [63]:
gru_pred = np.argmax(gru_model.predict(X_test_pad), axis=1)

print("\nGRU Classification Report:")
print(classification_report(
    y_test,
    gru_pred,
    target_names=["Negative", "Neutral", "Positive"]
))

556/556 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

GRU Classification Report:
              precision    recall  f1-score   support

    Negative       0.71      0.81      0.76      2664
     Neutral       0.67      0.43      0.52      4024
    Positive       0.84      0.93      0.88     11096

    accuracy                           0.80     17784
   macro avg       0.74      0.72      0.72     17784
weighted avg       0.78      0.80      0.78     17784



# **LSTM Model**

In [64]:
lstm_model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_shape=(MAX_LEN,)),
    LSTM(64),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation="softmax")
])

lstm_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

lstm_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,603 (5.07 MB)

 Trainable params: 1,329,603 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [65]:
start_time = time.time()

lstm_model.fit(
    X_train_pad,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

lstm_time = time.time() - start_time

Epoch 1/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6237 - loss: 0.9100 - val_accuracy: 0.6385 - val_loss: 0.8612
Epoch 2/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6527 - loss: 0.8464 - val_accuracy: 0.6348 - val_loss: 0.8412
Epoch 3/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.6654 - loss: 0.7743 - val_accuracy: 0.6747 - val_loss: 0.7553
Epoch 4/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.7051 - loss: 0.7128 - val_accuracy: 0.7056 - val_loss: 0.7104
Epoch 5/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.7469 - loss: 0.6235 - val_accuracy: 0.7496 - val_loss: 0.6097
Epoch 6/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.7956 - loss: 0.5190 - val_accuracy: 0.7746 - val_loss: 0.5797
Epoch 7/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8349 - loss: 0.4326 - val_accuracy: 0.7755 - val_loss: 0.5787
Epoch 8/50
890/890 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8593 - loss: 0.3735 - val_accuracy: 

In [66]:
lstm_pred = np.argmax(lstm_model.predict(X_test_pad), axis=1)

print("\nLSTM Classification Report:")
print(classification_report(
    y_test,
    lstm_pred,
    target_names=["Negative", "Neutral", "Positive"]
))

556/556 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

LSTM Classification Report:
              precision    recall  f1-score   support

    Negative       0.64      0.79      0.71      2664
     Neutral       0.59      0.47      0.52      4024
    Positive       0.86      0.87      0.87     11096

    accuracy                           0.77     17784
   macro avg       0.70      0.71      0.70     17784
weighted avg       0.77      0.77      0.77     17784



# **Test model**

In [67]:
test_sentences = [
    "this product is good",
    "this product is not good",
    "this product is bad",
    "this product is not bad",
    "not good at all",
    "not bad actually",
    "this is great",
    "this is not great",
]

def preprocess_sentences(sentences):
    sentences = [clean_text(s) for s in sentences]
    seq = tokenizer.texts_to_sequences(sentences)
    return pad_sequences(seq, maxlen=MAX_LEN, padding="post")

X_custom = preprocess_sentences(test_sentences)

gru_labels = gru_model.predict(X_custom).argmax(axis=1)
lstm_labels = lstm_model.predict(X_custom).argmax(axis=1)

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

print("Sentence".ljust(30), "GRU".ljust(12), "LSTM")
print("-"*55)

for i, s in enumerate(test_sentences):
    print(
        s.ljust(30),
        label_map[gru_labels[i]].ljust(12),
        label_map[lstm_labels[i]]
    )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Sentence                       GRU          LSTM
-------------------------------------------------------
this product is good           Positive     Positive
this product is not good       Negative     Negative
this product is bad            Negative     Negative
this product is not bad        Negative     Negative
not good at all                Negative     Negative
not bad actually               Negative     Negative
this is great                  Positive     Positive
this is not great              Negative     Positive


In [68]:
sentence = ["not bad"]

X_one = preprocess_sentences(sentence)

gru_prob = gru_model.predict(X_one)[0]
lstm_prob = lstm_model.predict(X_one)[0]

print("Sentence:", sentence[0])
print("\nGRU probabilities:")
print("Negative:", gru_prob[0])
print("Neutral :", gru_prob[1])
print("Positive:", gru_prob[2])

print("\nLSTM probabilities:")
print("Negative:", lstm_prob[0])
print("Neutral :", lstm_prob[1])
print("Positive:", lstm_prob[2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Sentence: not bad

GRU probabilities:
Negative: 0.86857754
Neutral : 0.10884786
Positive: 0.02257468

LSTM probabilities:
Negative: 0.8086735
Neutral : 0.14539935
Positive: 0.04592718
